In [ ]:
!pip install --upgrade category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 4.7 MB/s eta 0:00:00


#Creación del modelo de machine learning para prediccion de muertes prenatales usando el algoritmo de regresión logística


In [26]:
# Importamos librerías
import pandas as pd
from pandas import read_csv as read
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression as Regression 
from sklearn.tree import DecisionTreeClassifier as tree
from category_encoders import BinaryEncoder
from sklearn.metrics import accuracy_score, auc, f1_score ,recall_score, roc_curve
#librerias para guardar y cargar modelos
import pickle
import joblib

Cargamos Datos y asignamos columnas

In [22]:
#cargamos datos limpios
data = pd.read_csv("embarazos_medellin_limpios.csv", sep = ",")
display(data)

,ANO,MES,TIPO_PRODUCTO,SEXO,EDAD_MADRE,SEG_SOCIAL,COD_COMUNA_RES,APORTE_EMB,IDCLASADMI_0,IDCLASADMI_1,...,IDCLASADMI_7,BARRIO_RES_0,BARRIO_RES_1,BARRIO_RES_2,BARRIO_RES_3,BARRIO_RES_4,BARRIO_RES_5,BARRIO_RES_6,BARRIO_RES_7,BARRIO_RES_8
0,2012,11,1,3,41,1,15,1.0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,2012,4,1,3,26,1,3,1.0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,2012,3,1,3,36,1,12,1.0,0,0,...,1,0,0,0,0,0,0,0,1,1
3,2012,4,1,3,28,1,16,1.0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2012,6,1,3,36,1,7,1.0,0,0,...,1,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275062,2020,12,0,2,46,2,6,1.0,0,1,...,1,0,1,0,1,0,1,0,1,0
275063,2020,12,0,1,45,1,9,1.0,0,0,...,1,0,1,1,0,0,1,1,0,0
275064,2020,3,0,2,50,1,7,1.0,0,0,...,1,0,0,0,1,0,0,0,0,1
275065,2020,8,0,1,51,1,7,1.0,0,0,...,1,0,1,1,1,1,0,1,1,1


Separación de datos


In [23]:
# Separamos datos
Y= data[['TIPO_PRODUCTO']] #Target u objetivo
X = data.drop("TIPO_PRODUCTO", axis = 1)

# Separación en datos de entrenamiento y datos de validación 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7) 

# 70% de los datos son para entrenar 
# 30% de los datos son para validar

Creación del modelo de regresión logística

In [24]:
modelo = Regression() 
# Entrenamos al modelo
modelo.fit(X_train, Y_train)
modelo.coef_

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[-6.57036013e-03, -9.54197161e-03,  4.92480898e+00,
         1.66994239e-02,  1.54566413e-01,  4.16586264e-03,
        -5.01467143e-03, -1.46521440e-03, -2.05712152e-01,
        -5.13224292e-03, -2.92149643e-01,  8.56968055e-02,
        -7.89403759e-02,  2.38627439e-01, -2.61178644e-01,
        -5.53371821e-02, -1.05952008e-01, -6.85013455e-02,
         1.20533665e-02,  2.20186523e-02, -8.98422752e-02,
         1.75442118e-01,  2.95452130e-02, -8.03916575e-03]])

Análisis de calidad del modelo generado

In [25]:
Y_predict = modelo.predict(X_test) # Hacemos predicciones con los datos de test

# Obtenemos la precisión del modelo usando los datos predichos por el modelo y los datos reales
score_regression = accuracy_score(Y_test,Y_predict) # Usamos accuracy_score de sklearn metrics

# Obtener la sensibilidad del modelo, mide la proporción de verdaderos positivos que son identificados correctamente
sensibilidad = recall_score(Y_test,Y_predict) # Usamos recall_score de sklearn metrics

# Podemos usar f1-score que es una medida que combina la dos anteriores y mide la calidad global del modelo
score_f1 = f1_score(Y_test,Y_predict) # usamos f1_score de sklearn metrics

print(f"Precision = {score_regression} \nRecall = {sensibilidad}\nF1 = {score_f1}")

Precision = 0.9851431756764945 
Recall = 0.8514923562242174
F1 = 0.9196697680513695



MALO
Se puede observar que el modelo tiene una muy buena predicción debido a que su valor se acerca a 1, en otras palabras, el modelo predice correctamente a un 97%. Así mismo, el modelo es altamente sensible y puede diferencias entre falsos y verdaderos positivos aunque este valor de 0.82 también nos indica que existen ocasiones con error en la predicción. Sin embrago, el modelo tiene muy buena precision y sensibilidad para predecir sin errores. Analogamente, se obtuvo un buen valor para f1 (0.87) lo que nos indica que la calidad del modelo es eficiente.

In [27]:
#guardamos el modelo creado 
joblib.dump(modelo, 'modelo_entrenado_regresion_logistica.pkl')

['modelo_entrenado_regresion_logistica.pkl']